In [66]:
import matplotlib.pyplot as plt
import numpy as np
import sympy as sp
from sympy.physics.vector import dynamicsymbols

In [73]:
class Transformation:
    
    @staticmethod
    def rotate_x(angle: int | float | sp.Symbol) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[1:3, 1:3] = sp.Matrix([
            [sp.cos(angle), -sp.sin(angle)],
            [sp.sin(angle), sp.cos(angle)]
        ])
        return rot
    
    @staticmethod
    def rotate_z(angle: int | float | sp.Symbol) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[0:2, 0:2] = sp.Matrix([
            [sp.cos(angle), -sp.sin(angle)],
            [sp.sin(angle), sp.cos(angle)]
        ])
        return rot
    
    @staticmethod
    def rotate_y(angle: int | float | sp.Symbol) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[0, 0:3] = sp.Matrix([sp.cos(angle), 0, sp.sin(angle)]).T
        rot[2, 0:3] = sp.Matrix([-sp.sin(angle), 0, sp.cos(angle)]).T
        return rot
    
    @staticmethod
    def translate(pos: sp.Matrix | np.ndarray | list) -> sp.Matrix:
        rot = sp.eye(4, 4)
        rot[0:3, 3] = pos
        return rot
    
    
    def link(self, angle_z: int | float | sp.Symbol, dist_z: int | float | sp.Symbol, angle_x: int | float | sp.Symbol, dist_x: int | float | sp.Symbol) -> sp.Matrix:
        return self.rotate_z(angle_z) @ self.translate([0, 0, dist_z]) @ self.rotate_x(angle_x) @ self.translate([dist_x, 0, 0])


class Robot(Transformation):
    
    def __init__(self) -> None:
        self.links: list[sp.Matrix] = []
    
    def add_link(self, angle_z: int | float | sp.Symbol, dist_z: int | float | sp.Symbol, angle_x: int | float | sp.Symbol, dist_x: int | float | sp.Symbol) -> "Robot":
        link = self.rotate_z(angle_z) @ self.translate([0, 0, dist_z]) @ self.rotate_x(angle_x) @ self.translate([dist_x, 0, 0])
        self.links.append(link)
        return self
    
    def forward(self) -> sp.Matrix:
        T = sp.eye(4, 4)
        for link in self.links:
            T = T @ link
        return sp.simplify(T)

In [78]:
q = dynamicsymbols("q_{1:7}")
x, y, z = sp.symbols("x, y, z")

robot = Robot()
T = robot\
    .add_link(q[0], 0, 0, 0)\
    .add_link(q[1], 0, -sp.pi / 2, 0)\
    .add_link(q[2], sp.symbols("d_3"), 0, sp.symbols("a_2"))\
    .add_link(q[3], sp.symbols("d_4"), -sp.pi / 2, sp.symbols("a_3"))\
    .add_link(q[4], 0, sp.pi / 2, 0)\
    .add_link(q[5], 0, -sp.pi / 2, 0)\
    .forward()

In [84]:
sp.simplify(robot.links[-3] @ robot.links[-2] @ robot.links[-1])[0:3, 0:3]

Matrix([
[-sin(q_{4}(t))*sin(q_{6}(t)) + cos(q_{4}(t))*cos(q_{5}(t))*cos(q_{6}(t)), -sin(q_{5}(t))*cos(q_{4}(t)), -sin(q_{4}(t))*cos(q_{6}(t)) - sin(q_{6}(t))*cos(q_{4}(t))*cos(q_{5}(t))],
[ sin(q_{4}(t))*cos(q_{5}(t))*cos(q_{6}(t)) + sin(q_{6}(t))*cos(q_{4}(t)), -sin(q_{4}(t))*sin(q_{5}(t)), -sin(q_{4}(t))*sin(q_{6}(t))*cos(q_{5}(t)) + cos(q_{4}(t))*cos(q_{6}(t))],
[                                            -sin(q_{5}(t))*cos(q_{6}(t)),               -cos(q_{5}(t)),                                              sin(q_{5}(t))*sin(q_{6}(t))]])

In [87]:
def check_euler(q1, q2, q3, pattern):
    T = sp.eye(4, 4)
    for a, l in zip([q1, q2, q3], list(pattern)):
        if l == "x":
            T = T @ Transformation.rotate_x(a)
        if l == "y":
            T = T @ Transformation.rotate_y(a)
        if l == "z":
            T = T @ Transformation.rotate_z(a)
    return sp.simplify(T)[0:3, 0:3]

In [109]:
check_euler(q[-3], q[-2], q[-1], "zxz")

Matrix([
[-sin(q_{4}(t))*sin(q_{6}(t))*cos(q_{5}(t)) + cos(q_{4}(t))*cos(q_{6}(t)), -sin(q_{4}(t))*cos(q_{5}(t))*cos(q_{6}(t)) - sin(q_{6}(t))*cos(q_{4}(t)),  sin(q_{4}(t))*sin(q_{5}(t))],
[ sin(q_{4}(t))*cos(q_{6}(t)) + sin(q_{6}(t))*cos(q_{4}(t))*cos(q_{5}(t)), -sin(q_{4}(t))*sin(q_{6}(t)) + cos(q_{4}(t))*cos(q_{5}(t))*cos(q_{6}(t)), -sin(q_{5}(t))*cos(q_{4}(t))],
[                                             sin(q_{5}(t))*sin(q_{6}(t)),                                              sin(q_{5}(t))*cos(q_{6}(t)),                cos(q_{5}(t))]])

In [111]:
T[:3, 3]

Matrix([
[a_2*cos(q_{1}(t) + q_{2}(t))*cos(q_{3}(t)) + a_3*cos(q_{1}(t) + q_{2}(t))*cos(q_{3}(t) + q_{4}(t)) - d_3*sin(q_{1}(t) + q_{2}(t)) - d_4*sin(q_{1}(t) + q_{2}(t))],
[a_2*sin(q_{1}(t) + q_{2}(t))*cos(q_{3}(t)) + a_3*sin(q_{1}(t) + q_{2}(t))*cos(q_{3}(t) + q_{4}(t)) + d_3*cos(q_{1}(t) + q_{2}(t)) + d_4*cos(q_{1}(t) + q_{2}(t))],
[                                                                                                               -a_2*sin(q_{3}(t)) - a_3*sin(q_{3}(t) + q_{4}(t))]])